# Lab 10 -- Decision Trees for Regression

Compared to last week, this is a very simple lab <span style="font-size:20pt;">😃</span> You'll have fun programming!

You will implement the **Classification and Regression Tree (CART)** algorithm from scratch.

The lab is broken down into the following pieces:

* Regression Criterion
* Creating Splits
* Buiding a Tree
* Making a prediction

## Exercise 1 -- Download and load the dataset

We will be using the usual Boston Housing dataset, which is available to download from ECLASS

* Download the file
* Read it and separate the target variable from the features.
* Make a 80/10/10 train/validation/test split

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
path = 'housing.txt'
boston = pd.read_csv(path,  delim_whitespace=True)


C:\Users\Matheus Pinto\AppData\Local\Temp\ipykernel_8672\2965622961.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  boston = pd.read_csv(path,  delim_whitespace=True)


The target variable will be as usual `MEDV`. Use the rest as features.

In [149]:
X = boston.values[:,:-1]
y = boston.iloc[:,-1:]
X = np.array(X, dtype=float)
y = np.array(y, dtype=float)
X_tv, X_test, y_tv, y_test = train_test_split(X, y,test_size=0.10, random_state=10, shuffle=True)
X_train, X_validation, y_train, y_validation = train_test_split(X_tv, y_tv, test_size=0.10/0.80 , random_state=300, shuffle=True) 



print(f"Training Size: {len(X_train) / len(X)}")
print(f"Validation Size: {len(X_validation) / len(X)}")
print(f"Test Size: {len(X_test) / len(X)}")


Training Size: 0.7861386138613862
Validation Size: 0.11287128712871287
Test Size: 0.100990099009901


## Exercise 2 -- Optimization Criterion

For regression, a simple criterion to optimize is to minimize the sum of squared errors for a given region. This is, for all datapoints in a region with size, we minimize:

$$\sum_{i=1}^N(y_i - \hat{y})^2$$

where $N$ is the number of datapoits in the region and $\hat{y}$ is the mean value of the region for the target variable. 

Implement such a function using the description below.

Please, don't use an existing implementation, refer to the [book](https://www.statlearning.com/s/ISLRSeventhPrinting.pdf), and if you need help, ask questions!

In [6]:
# # your code here
# housing = pd.read_csv(path,  delim_whitespace=True)
# X = housing.values[:,:-1]
# y = housing.values[:,-1:]

# X_train, X_test_valid, y_train, y_test_valid = train_test_split(X, y, train_size=0.8, random_state=42, shuffle=True)
# X_valid, X_test, y_valid, y_test = train_test_split(X_test_valid, y_test_valid, train_size=0.5, random_state=42,shuffle=True)

In [7]:
def regression_criterion(region):
    """
    Implements the sum of squared error criterion in a region
    
    Parameters
    ----------
    region : ndarray
        Array of shape (N,) containing the values of the target values 
        for N datapoints in the training set.
    
    Returns
    -------
    float
        The sum of squared error
        
    Note
    ----
    The error for an empty region should be infinity (use: float("inf"))
    This avoids creating empty regions
    """
    if region.shape == (0,):
        return float("inf")
    return np.sum((region - np.mean(region)) ** 2)

In [8]:
# test your code
rng = np.random.default_rng(0)
print(regression_criterion(rng.random(size=40)))
print(regression_criterion(np.ones(10)))
print(regression_criterion(np.zeros(10)))
print(regression_criterion(np.array([])))

3.6200679838629544
0.0
0.0
inf


## Exercise 3 -- Make a split

In [ ]:
def split_region(region, feature_index, tau):
    """
    Given a region, splits it based on the feature indicated by
    `feature_index`, the region will be split in two, where
    one side will contain all points with the feature with values 
    lower than `tau`, and the other split will contain the 
    remaining datapoints.
    
    Parameters
    ----------
    region : array of size (n_samples, n_features)
        a partition of the dataset (or the full dataset) to be split
    feature_index : int
        the index of the feature (column of the region array) used to make this partition
    tau : float
        The threshold used to make this partition
        
    Return
    ------
    left_partition : array
        indices of the datapoints in `region` where feature < `tau`
    right_partition : array
        indices of the datapoints in `region` where feature >= `tau` 
    """
    l = np.where(region[:,feature_index] < tau)[0]
    r = np.where(region[:,feature_index] >= tau)[0]
    
    return l, r
     

In [10]:
l, r = split_region(X_train, 2, 10)
print(l.shape, r.shape)

(214,) (183,)


## Exercise 4 -- Find the best split

The strategy is quite simple (as well as inefficient), but it helps to reinforce the concepts.
We are going to use a greedy, exhaustive algorithm to select splits, selecting the `feature_index` and the `tau` that minimizes the Regression Criterion

In [11]:
def get_split(X, y):
    """
    Given a dataset (full or partial), splits it on the feature of that minimizes the sum of squared error
    
    Parameters
    ----------
    X : array (n_samples, n_features)
        features 
    y : array (n_samples, )
        labels
    
    Returns
    -------
    decision : dictionary
        keys are:
        * 'feature_index' -> an integer that indicates the feature (column) of `X` on which the data is split
        * 'tau' -> the threshold used to make the split
        * 'left_region' -> array of indices where the `feature_index`th feature of X is lower than `tau`
        * 'right_region' -> indices not in `low_region`
    """
    best_sse = float("inf")
    best_tau = 0
    best_feature_index = 0
    for feature_index in range(X.shape[1]):
        tau_list = np.unique(X[:, feature_index])
        for tau in tau_list:
            l, r = split_region(X, feature_index, tau)
            if len(l) == 0 or len(r) == 0:
                continue
            yl, rl = y[l], y[r]
            sse = regression_criterion(yl) + regression_criterion(rl)
            if sse < best_sse:
                best_sse = sse
                best_tau = tau
                best_feature_index = feature_index

    if best_sse == float("inf"):
        return {
            'feature_index': 0,
            'tau': 0,
            'left_region': np.array([]),
            'right_region': np.arange(X.shape[0])
        }
    left_region, right_region = split_region(X, best_feature_index, best_tau)
    dic = {
        'feature_index': best_feature_index,
        'tau': best_tau,
        'left_region': left_region,
        'right_region': right_region,
    }

    return dic



In [12]:
get_split(X_train[:15, :], y_train[:15])

{'feature_index': 5,
 'tau': np.float64(7.249),
 'left_region': array([ 0,  1,  2,  3,  5,  6,  7,  8,  9, 11, 12, 14]),
 'right_region': array([ 4, 10, 13])}

## Exercise 5 -- Recursive Splitting

The test above is an example on how to find the root node of our decision tree. The algorithm now is a greedy search until we reach a stop criterion. To find the actual root node of our decision tree, you must provide the whole training set, not just a slice of 15 rows as the test above.

The trivial stopping criterion is to recursively grow the tree until each split contains a single point (perfect node purity). If we go that far, it normally means we are overfitting.

You will implement these criteria to stop the growth:

* A node is a leaf if:
    * It has less than `min_samples` datapoints
    * It is at the `max_depth` level from the root (each split creates a new level)
    * The criterion is `0`



In [134]:
def recursive_growth(node, min_samples, max_depth, current_depth, X, y):
    """
    Recursively grows a decision tree.
    
    Parameters
    ----------
    node : dictionary
        If the node is terminal, it contains only the "value" key, which determines the value to be used as a prediction.
        If the node is not terminal, the dictionary has the structure defined by `get_split`
    min_samples : int
        parameter for stopping criterion if a node has <= min_samples datapoints
    max_depth : int
        parameter for stopping criterion if a node belongs to this depth
    depth : int
        current distance from the root
    X : array (n_samples, n_features)
        features (full dataset)
    y : array (n_samples, )
        labels (full dataset)
    
    Notes
    -----
    To create a terminal node, a dictionary is created with a single "value" key, with a value that
    is the mean of the target variable
    
    'left' and 'right' keys are added to non-terminal nodes, which contain (possibly terminal) nodes 
    from higher levels of the tree:
    'left' corresponds to the 'left_region' key, and 'right' to the 'right_region' key
    """
    region = np.concatenate((node['left_region'],node['right_region']))
    if len(region) <= min_samples or current_depth >= max_depth or regression_criterion(y[region]) == 0:
        node.clear()
        region = region.astype(dtype=np.int16)
        node['value'] = y[region].mean()
        return

    # for the left node:
    left_indices = node['left_region']
    X_left = X[left_indices]
    y_left = y[left_indices]
    node['left_region'] = get_split(X_left, y_left)
    recursive_growth(node['left_region'], min_samples, max_depth, current_depth + 1, X_left, y_left)

    # for the right node:
    right_indices = node['right_region']
    X_right = X[right_indices]
    y_right = y[right_indices]
    node['right_region'] = get_split(X_right, y_right)
    recursive_growth(node['right_region'], min_samples, max_depth, current_depth + 1, X_right, y_right)

    node['right'] = node['right_region']
    node['left'] = node['left_region']


In [61]:
k = 20
test_root = get_split(X_train[:k, :], y_train[:k])
recursive_growth(test_root, 5, 3, 1, X_train[:k, :], y_train[:k])

Below we provide code to visualise the generated tree!

In [96]:
def print_tree(node, depth):
    if 'value' in node.keys():
        print('.  '*(depth-1), f"[{node['value']}]")
    else:
        print('.  '*depth, f'X_{node["feature_index"]} < {node["tau"]}')
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)


In [135]:
print_tree(root, 0)

 X_12 < 9.8
.   X_12 < 16.14
.  .   X_4 < 0.609
.  .  .   X_12 < 19.77
.  .  .  .   X_9 < 666.0
.  .  .  .   [9.864102564102565]
.  .  .  .   [13.957142857142857]
.  .  .  .   X_0 < 13.3598
.  .  .  .   [10.9]
.  .  .  .   [15.465517241379313]
.  .  .   X_10 < 21.0
.  .  .   [13.9125]
.  .  .  .   X_5 < 6.852
.  .  .  .   [27.5]
.  .  .  .   [18.655172413793103]
.  .   X_4 < 0.671
.  .   [17.46315789473684]
.  .  .   X_5 < 6.879
.  .  .   [26.833333333333332]
.  .  .  .   X_2 < 2.89
.  .  .  .   [20.68488372093023]
.  .  .  .   [25.366666666666664]
.   X_5 < 6.943
.  .   X_5 < 7.454
.  .   [44.555]
.  .  .   X_7 < 1.8946
.  .  .  .   X_2 < 11.93
.  .  .  .   [24.45]
.  .  .  .   [33.45142857142857]
.  .  .   [45.65]
.  .   X_0 < 4.89822
.  .   [50.0]
.  .  .   X_5 < 6.546
.  .  .  .   X_12 < 4.7
.  .  .  .   [27.066666666666666]
.  .  .  .   [31.03]
.  .  .  .   X_9 < 223.0
.  .  .  .   [22.795454545454547]
.  .  .  .   [36.2]


# Exercise 6 -- Make a Prediction
Use the a node to predict the class of a compatible dataset

In [106]:
def predict_sample(node, sample):
    """
    Makes a prediction based on the decision tree defined by `node`
    
    Parameters
    ----------
    node : dictionary
        A node created one of the methods above
    sample : array of size (n_features,)
        a sample datapoint
    """
    if 'value' in node:
        return node['value']
    
    if sample[node['feature_index']] < node['tau']:
        return predict_sample(node['left'], sample)
    else:
        return predict_sample(node['right'], sample)
        
def predict(node, X):
    """
    Makes a prediction based on the decision tree defined by `node`
    
    Parameters
    ----------
    node : dictionary
        A node created one of the methods above
    X : array of size (n_samples, n_features)
        n_samples predictions will be made
    """
    predictions = []
    for s in X:
        p = predict_sample(node, s)
        predictions.append(p)
    return np.array(predictions)

Now use the functions defined above to calculate the RMSE of the validation set. 
* Try first with `min_samples=20` and `max_depth=6` (for this values you should get a validation RMSE of ~8.8)

Then, experiment with different values for the stopping criteria.

In [150]:
# fill in the gaps with your code
min_samples = 20
max_depth = 6
root = get_split(X_train, y_train)
recursive_growth(root, min_samples, max_depth,1,X_train, y_train)

y_p = predict(root, X_validation)

rmse = np.sqrt(np.mean((y_validation - y_p) ** 2))
print(rmse)

13.765585354146216
